In [ ]:
import os
import rasterio
import numpy as np
from skimage import measure
from concurrent.futures import ThreadPoolExecutor

def calculate_forest_area_and_boundary_length(tif_path, output_path, block_size=512):
    with rasterio.open(tif_path) as src:
        forest_area = np.zeros((10, 10))
        boundary_length = np.zeros((10, 10))

        for ji, window in src.block_windows(1):
            block = src.read(1, window=window)
            
            block_window = src.window_transform(window)
            row_start, col_start = int(window.row_off // (src.height / 10)), int(window.col_off // (src.width / 10))

            for i in range(block.shape[0]):
                for j in range(block.shape[1]):
                    row, col = row_start + i // block_size, col_start + j // block_size
                    if block[i, j] == 1:
                        forest_area[row, col] += 1
                        if i > 0 and block[i - 1, j] != 1:
                            boundary_length[row, col] += src.res[0]
                        if j > 0 and block[i, j - 1] != 1:
                            boundary_length[row, col] += src.res[1]
                            
        # Convert pixel counts to actual area
        forest_area *= src.res[0] * src.res[1]

        # Save results to file
        np.savetxt(os.path.join(output_path, f'{os.path.splitext(os.path.basename(tif_path))[0]}_area.txt'), forest_area)
        np.savetxt(os.path.join(output_path, f'{os.path.splitext(os.path.basename(tif_path))[0]}_boundary.txt'), boundary_length)

def process_file(file, folder_2000, folder_2020, output2000, output2020):
    print(f'Processing {file}...')
    path_2000 = os.path.join(folder_2000, file)
    path_2020 = os.path.join(folder_2020, file)

    calculate_forest_area_and_boundary_length(path_2000, output2000)
    print('Results saved to:', output2000)

    calculate_forest_area_and_boundary_length(path_2020, output2020)
    print('Results saved to:', output2020)

folder_2000 = 'H:/Global_tree_cover/2000extent'
folder_2020 = 'H:/Global_tree_cover/2020extent'

output2000 = 'H:/Global_tree_cover/output2000'
output2020 = 'H:/Global_tree_cover/output2020'

file_list = os.listdir(folder_2000)

# Set the number of worker threads in the ThreadPoolExecutor
num_workers = 8

# Process the files in parallel using a ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=num_workers) as executor:
    futures = [executor.submit(process_file, file, folder_2000, folder_2020, output2000, output2020) for file in file_list]
    for future in futures:
        future.result()

Processing 30N_110E.tif...
Processing 30N_110W.tif...
Processing 30N_120E.tif...
Processing 30N_120W.tif...
Processing 30N_130E.tif...
Processing 30N_160W.tif...
Processing 30N_170W.tif...
Processing 30S_010E.tif...
Results saved to: H:/Global_tree_cover/output2000
Results saved to: H:/Global_tree_cover/output2000
Results saved to: H:/Global_tree_cover/output2000
Results saved to: H:/Global_tree_cover/output2000
Results saved to: H:/Global_tree_cover/output2000
Results saved to: H:/Global_tree_cover/output2000
Results saved to: H:/Global_tree_cover/output2000
Results saved to: H:/Global_tree_cover/output2000
Results saved to: H:/Global_tree_cover/output2020
Processing 30S_020E.tif...
Results saved to: H:/Global_tree_cover/output2020
Processing 30S_030E.tif...
Results saved to: H:/Global_tree_cover/output2020
Processing 30S_060W.tif...
Results saved to: H:/Global_tree_cover/output2000
Results saved to: H:/Global_tree_cover/output2000
Results saved to: H:/Global_tree_cover/output2020
Pro